### LangChain Intro — Local Ollama (Qwen)

This notebook uses **Ollama** with a local model. **No API key or .env file is used or required.**

**Prerequisites:** Ollama running locally and at least one model installed.  
Run `ollama list` in a terminal to see installed models. Set `OLLAMA_MODEL` in the next cell to the **NAME** from that list (e.g. `qwen2.5-coder:14b-instruct-q8_0` or `llama3.2:latest`).

In [3]:
# No .env or API key needed for Ollama — it runs fully locally.
from langchain.agents import create_agent
from langchain_ollama import ChatOllama

# Use the exact NAME from `ollama list` (e.g. qwen2.5-coder:14b-instruct-q8_0, llama3.2:latest)
OLLAMA_MODEL = "qwen2.5-coder:14b-instruct-q8_0"
ollama_llm = ChatOllama(model=OLLAMA_MODEL, temperature=0.7)

agent_ollama = create_agent(
    model=ollama_llm,
    tools=[],
    system_prompt="You are helpful.",
)
print("Ollama agent created (no API key used). Invoke with: agent_ollama.invoke({'messages': [...]})")

Ollama agent created (no API key used). Invoke with: agent_ollama.invoke({'messages': [...]})


In [4]:
# Test the basic Ollama agent (use "messages" as the input key)
try:
    out = agent_ollama.invoke({"messages": [{"role": "user", "content": "Say hello in one sentence."}]})
    last = out.get("messages", [])[-1]
    print(last.content if hasattr(last, "content") and isinstance(last.content, str) else out)
except Exception as e:
    err = str(e).lower()
    if "not found" in err or "404" in err:
        print("Model not found. Run 'ollama list' and set OLLAMA_MODEL in the cell above to one of the NAME values.")
    raise

Hello!


#### Agent with weather tool

Same `get_weather` tool as in the Amdocs notebook; agent can use it for weather questions.

In [5]:
def get_weather(city: str) -> str:
    """Get the weather in a city."""
    return f"The weather in {city} is sunny."

In [6]:
agent_ollama_weather = create_agent(
    model=ollama_llm,
    tools=[get_weather],
    system_prompt="You are helpful. Use the get_weather tool when the user asks about weather in a city.",
)
print("Ollama agent with weather tool created.")

Ollama agent with weather tool created.


In [ ]:
# Invoke with a weather question (input key must be "messages")
try:
    out = agent_ollama_weather.invoke({
        "messages": [{"role": "user", "content": "What is the weather in Mumbai?"}]
    })
    last = out.get("messages", [])[-1]
    print(last.content if hasattr(last, "content") and isinstance(last.content, str) else out)
except Exception as e:
    if "not found" in str(e).lower():
        print("Model not found. Set OLLAMA_MODEL in the first code cell to a name from 'ollama list'.")
    raise

{
  "name": "get_weather",
  "arguments": {
    "city": "Mumbai"
  }
}


: 